In [ ]:
import os

required_files = ['model.py', 'lightning_trainer.py', 'train_colab.py', 'input.txt', 'requirements.txt']
missing_files = [f for f in required_files if not os.path.exists(f)]

if missing_files:
    print("❌ Missing files:")
    for f in missing_files:
        print(f"   - {f}")
    print("\nPlease upload these files before continuing!")
else:
    print("✅ All required files found!")
    print("\nFiles:")
    !ls -lh model.py lightning_trainer.py train_colab.py input.txt requirements.txt

✅ All required files found!

Files:
-rw-r--r-- 1 root root 1.1M Oct 20 15:22 input.txt
-rw-r--r-- 1 root root  12K Oct 20 15:22 lightning_trainer.py
-rw-r--r-- 1 root root  17K Oct 20 15:22 model.py
-rw-r--r-- 1 root root  652 Oct 20 15:23 requirements.txt
-rw-r--r-- 1 root root 8.9K Oct 20 15:22 train_colab.py


In [ ]:
# Install PyTorch with CUDA support
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

# Install other dependencies (including transformers for the BPE tokenizer)
!pip install -q pytorch-lightning transformers gradio tensorboard

print("✅ Installation complete!")
print("\nNote: This includes transformers for the official SmolLM2 tokenizer")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 832.4/832.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 61.0 MB/s eta 0:00:00
✅ Installation complete!

Note: This includes transformers for the official SmolLM2 tokenizer


In [ ]:
import torch
import pytorch_lightning as pl

print("="*60)
print("System Information")
print("="*60)
print(f"PyTorch version: {torch.__version__}")
print(f"Lightning version: {pl.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"\n✅ GPU detected!")
    print(f"   Device: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
    print(f"   CUDA Version: {torch.version.cuda}")
else:
    print("\n⚠️ No GPU detected!")
    print("Please enable GPU: Runtime → Change runtime type → T4 GPU")
    print("Then restart the runtime and run again.")

print("="*60)

System Information
PyTorch version: 2.8.0+cu126
Lightning version: 2.5.5
CUDA available: True

✅ GPU detected!
   Device: Tesla T4
   Memory: 15.83 GB
   CUDA Version: 12.6


In [ ]:
from model import SmolLM2ForCausalLM, get_model_config, count_parameters
from transformers import AutoTokenizer

print("Testing model creation with official tokenizer...")

# Load official SmolLM2 tokenizer
tokenizer = AutoTokenizer.from_pretrained('HuggingFaceTB/SmolLM2-135M')
print(f"✅ Tokenizer loaded: {type(tokenizer).__name__}")
print(f"   Vocab size: {tokenizer.vocab_size:,}")

# Create model with correct vocab
config = get_model_config()
model = SmolLM2ForCausalLM(config)
param_info = count_parameters(model)

print(f"\n✅ Model created successfully!")
print(f"   Total parameters: {param_info['total_millions']:.2f}M (should be ~135M)")
print(f"   Vocab size: {config['vocab_size']:,}")
print(f"   Hidden size: {config['hidden_size']}")
print(f"   Num layers: {config['num_hidden_layers']}")
print(f"   Attention heads: {config['num_attention_heads']}")
print(f"   KV heads: {config['num_key_value_heads']}")

# Clean up
del model
torch.cuda.empty_cache()
print("\n✅ Ready for training with official SmolLM2 tokenizer!")

Testing model creation with official tokenizer...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: ec3db9ab-07c9-4ffe-b672-ca977023e73f)')' thrown while requesting HEAD https://huggingface.co/HuggingFaceTB/SmolLM2-135M/resolve/main/tokenizer_config.json
Retrying in 1s [Retry 1/5].


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 81b5155b-71de-4ac0-a90a-79266a619ffa)')' thrown while requesting HEAD https://huggingface.co/HuggingFaceTB/SmolLM2-135M/resolve/main/tokenizer.json
Retrying in 1s [Retry 1/5].


tokenizer.json: 0.00B [00:00, ?B/s]

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 3bb99949-b0fb-4d1d-ae07-3aed645cc5fd)')' thrown while requesting HEAD https://huggingface.co/HuggingFaceTB/SmolLM2-135M/resolve/main/special_tokens_map.json
Retrying in 1s [Retry 1/5].


special_tokens_map.json:   0%|          | 0.00/831 [00:00<?, ?B/s]

✅ Tokenizer loaded: GPT2TokenizerFast
   Vocab size: 49,152

✅ Model created successfully!
   Total parameters: 134.52M (should be ~135M)
   Vocab size: 49,152
   Hidden size: 576
   Num layers: 30
   Attention heads: 9
   KV heads: 3

✅ Ready for training with official SmolLM2 tokenizer!


In [9]:
# Option 2: Train from scratch (random initialization)
!python train_colab.py --from-scratch

# Option 1: Fine-tune from pretrained (uncomment to use instead)
# !python train_colab.py --pretrained

Seed set to 42
SmolLM2-135M Training on Shakespeare Dataset

[1/6] Loading dataset...
Dataset loaded: 1,115,394 characters

[2/6] Loading SmolLM2-135M tokenizer from HuggingFace...
✓ Set pad_token to eos_token: <|endoftext|>
✓ Tokenizer loaded:
  - Type: GPT2TokenizerFast
  - Vocab size: 49,152
  - BOS/EOS token: <|endoftext|> (ID: 0)
  - Example: 100 chars → 31 tokens

[3/6] Creating dataloaders...

Dataset split:
  Training characters: 1,003,854
  Validation characters: 111,540
Token indices sequence length is longer than the specified maximum sequence length for this model (305111 > 8192). Running this sequence through the model will result in indexing errors
Total tokens: 305,111
Tokenizer vocab size: 49,152
Example: First 100 chars -> 31 tokens
Total tokens: 35,983
Tokenizer vocab size: 49,152
Example: First 100 chars -> 40 tokens
Training batches: 19054
Validation batches: 2233

[4/6] Initializing SmolLM2-135M model...
🎲 Training from scratch: Random initialization...
Initializin

In [10]:
import torch
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, LearningRateMonitor
from transformers import AutoTokenizer
from lightning_trainer import SmolLM2Lightning, create_dataloaders

# Configuration
BATCH_SIZE = 16
SEQ_LENGTH = 256
ADDITIONAL_STEPS = 50  # Train for exactly 50 more steps
GENERATE_EVERY_N_STEPS = 500
CHECKPOINT_PATH = '/content/checkpoints/last.ckpt'
TOKENIZER_NAME = 'HuggingFaceTB/SmolLM2-135M'

print("="*80)
print("Loading checkpoint and training for 50 more steps")
print("="*80)

# Load tokenizer
print("\n[1/4] Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
tokenizer.pad_token = tokenizer.eos_token
print(f"✓ Tokenizer loaded: {tokenizer.__class__.__name__}")

# Load data
print("\n[2/4] Loading dataset...")
with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(f"Dataset loaded: {len(text):,} characters")

# Create dataloaders
print("\n[3/4] Creating dataloaders...")
train_loader, val_loader = create_dataloaders(
    text=text,
    tokenizer=tokenizer,
    batch_size=BATCH_SIZE,
    seq_length=SEQ_LENGTH,
    train_split=0.9
)
print(f"Training batches: {len(train_loader)}")
print(f"Validation batches: {len(val_loader)}")

# Load model from checkpoint
print("\n[4/4] Loading model from checkpoint...")
model = SmolLM2Lightning.load_from_checkpoint(
    CHECKPOINT_PATH,
    tokenizer=tokenizer,
    learning_rate=3e-4,
    weight_decay=0.1,
    warmup_steps=500,
    generate_every_n_steps=GENERATE_EVERY_N_STEPS,
    from_scratch=True
)
print(f"✓ Checkpoint loaded from: {CHECKPOINT_PATH}")
print(f"✓ Model has {sum(p.numel() for p in model.parameters()):,} parameters")

# Get the current global step
checkpoint = torch.load(CHECKPOINT_PATH)
current_step = checkpoint['global_step']
print(f"\nResumed from step {current_step}")
print(f"Will train to step {current_step + ADDITIONAL_STEPS}")

# Setup trainer
checkpoint_callback = ModelCheckpoint(
    dirpath='./checkpoints',
    filename='continued-{step}',
    save_top_k=1,
    save_last=True,
    monitor='val_loss',
    mode='min'
)

lr_monitor = LearningRateMonitor(logging_interval='step')

trainer = pl.Trainer(
    max_steps=current_step + ADDITIONAL_STEPS,  # 5000 + 50 = 5050
    val_check_interval=ADDITIONAL_STEPS,
    log_every_n_steps=10,
    precision='bf16-mixed',
    callbacks=[checkpoint_callback, lr_monitor],
    enable_progress_bar=True,
    gradient_clip_val=1.0,
    accumulate_grad_batches=1,
)

# Train for 50 more steps
print("\n" + "="*80)
print(f"Starting training for {ADDITIONAL_STEPS} more steps...")
print("="*80 + "\n")

trainer.fit(model, train_loader, val_loader)

print("\n" + "="*80)
print("Training completed! 🎉")
print("="*80)

# Generate final sample
print("\nGenerating final sample text:")
print("="*80)
model.generate_and_log()
print("="*80)

print(f"\n✓ Final checkpoint saved")
print(f"✓ Training completed at step {current_step + ADDITIONAL_STEPS}")

Loading checkpoint and training for 50 more steps

[1/4] Loading tokenizer...
✓ Tokenizer loaded: GPT2TokenizerFast

[2/4] Loading dataset...
Dataset loaded: 1,115,394 characters

[3/4] Creating dataloaders...

Dataset split:
  Training characters: 1,003,854
  Validation characters: 111,540


Token indices sequence length is longer than the specified maximum sequence length for this model (305111 > 8192). Running this sequence through the model will result in indexing errors


Total tokens: 305,111
Tokenizer vocab size: 49,152
Example: First 100 chars -> 31 tokens
Total tokens: 35,983
Tokenizer vocab size: 49,152
Example: First 100 chars -> 40 tokens
Training batches: 19054
Validation batches: 2233

[4/4] Loading model from checkpoint...
Initializing model from scratch...
✓ Model initialized with random weights
📊 Model Parameters: 134,515,008
Resumed from step 5000
✓ Checkpoint loaded from: /content/checkpoints/last.ckpt
✓ Model has 134,515,008 parameters


INFO:pytorch_lightning.utilities.rank_zero:Using bfloat16 Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs



Resumed from step 5000
Will train to step 5050

Starting training for 50 more steps...



...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/call.py", line 49, in _call_and_handle_interrupt
    return trainer_fn(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 598, in _fit_impl
    self._run(model, ckpt_path=ckpt_path)
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1011, in _run
    results = self._run_stage()
              ^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/trainer/trainer.py", line 1055, in _run_stage
    self.fit_loop.run()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py", line 216, in run
    self.advance()
  File "/usr/local/lib/python3.12/dist-packages/pytorch_lightning/loops/fit_loop.py", line 458, in advance
    self.epoch_loop.run(self._data_fetcher)
  File "/usr/local/lib/python3.12

TypeError: object of type 'NoneType' has no len()

In [11]:
import torch
from transformers import AutoTokenizer
from lightning_trainer import SmolLM2Lightning

# Load your trained checkpoint
CHECKPOINT_PATH = '/content/checkpoints/continued-step=50.ckpt'
TOKENIZER_NAME = 'HuggingFaceTB/SmolLM2-135M'

print("Loading trained model...")
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_NAME)
tokenizer.pad_token = tokenizer.eos_token

model = SmolLM2Lightning.load_from_checkpoint(
    CHECKPOINT_PATH,
    tokenizer=tokenizer,
    learning_rate=3e-4,
    weight_decay=0.1,
    warmup_steps=500,
    generate_every_n_steps=500,
    from_scratch=True
)
model.eval()

checkpoint = torch.load(CHECKPOINT_PATH)
print(f"✓ Model loaded from step {checkpoint['global_step']}")
print(f"✓ Parameters: {sum(p.numel() for p in model.parameters()):,}")

# Generate Shakespeare-style text
print("\n" + "="*80)
print("Generating text with your trained model...")
print("="*80)

prompts = ["First Citizen:", "ROMEO:", "To be or not to be"]

for prompt in prompts:
    print(f"\n📝 Prompt: '{prompt}'")
    print("-"*80)

    input_ids = tokenizer.encode(prompt, add_special_tokens=False, return_tensors='pt')

    with torch.no_grad():
        generated_ids = model.model.generate(
            input_ids,
            max_new_tokens=150,
            temperature=0.8,
            top_p=0.9,
            do_sample=True,
            pad_token_id=tokenizer.pad_token_id,
        )

    text = tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    print(text)
    print("-"*80)

print("\n✅ Done! Your model is trained and working!")

Loading trained model...
Initializing model from scratch...
✓ Model initialized with random weights
📊 Model Parameters: 134,515,008
Resumed from step 5050
✓ Model loaded from step 50
✓ Parameters: 134,515,008

Generating text with your trained model...

📝 Prompt: 'First Citizen:'
--------------------------------------------------------------------------------


TypeError: SmolLM2ForCausalLM.generate() got an unexpected keyword argument 'max_new_tokens'